<a href="https://colab.research.google.com/github/krishnarane2005/Decentralized-login-system/blob/main/Ghibli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!nvidia-smi

Tue Apr  8 19:08:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P0             26W /   70W |    5446MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [56]:
import torch
print(f"GPU Available: {torch.cuda.is_available()}")  # Should return True
print(f"GPU Name: {torch.cuda.get_device_name(0)}")   # Should show "Tesla T4"

GPU Available: True
GPU Name: Tesla T4


In [57]:
!pip install -q diffusers accelerate transformers torch xformers
!pip install -q datasets pillow

In [58]:
from google.colab import files
uploaded = files.upload()

Saving IMG_3822.HEIC to IMG_3822.HEIC


In [59]:
!ls  "/content/drive/MyDrive/"

 Advanced_Problems_in_Organic_Chemistry_for_JEE_by_M_S_Chouhan_15th.pdf
'Binomial Expression.pdf'
'Colab Notebooks'
'College material'
 convert_images
'Copy of EmployeeData.csv'
 DOC-20240312-WA0001..gdoc
 DOC-20240312-WA0001..pdf
 DSC_7766.jpg
'fc center confirmation.pdf'
'Final Merit Status.pdf'
 frontend
'indrayani report.gdoc'
'IOT PROJECT REPORT.gdoc'
 KRISHNA1.pdf
'KRISHNARANE-Introduction to -certificate.pdf'
 KRISHNARANE-VIP2024-Cybersec-certificate.pdf
'KRISHNARANE-VIP2024-Packet T-certificate.pdf'
 NC.JPG
 photocopy.jpg
 practice
'_Premika_Ne_Pyar_Se_Bgm_Instrumental_Music_Ringtone_(by Fringster.com).mp3'
'Progression and Series.pdf'
'report format.gdoc'
'Screenshot 2024-08-18 180430.png'
' Shiv jayanti'
'Trigonometric Equations.pdf'
'TRIGONOMETRIC Function.pdf'
'Untitled folder'
'WhatsApp Image 2024-10-27 at 10.47.21 AM (1).jpeg'


In [60]:
# !ls "/content/drive/MyDrive/"
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# Move images to Colab workspace
!cp -r "/content/drive/MyDrive/convert_images" "/content/"


Mounted at /content/drive


In [61]:
!pip install -q git+https://github.com/huggingface/diffusers
!pip install -q accelerate transformers torch xformers
!pip install -q datasets pillow

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [63]:
from diffusers import DiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.training_utils import EMAModel
import torch
from torch.utils.data import Dataset
import os
from PIL import Image

# 1. Prepare Dataset
class GhibliDataset(Dataset):
    def __init__(self, image_dir, size=512):
        self.image_dir = image_dir
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]
        self.size = size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        img = img.resize((self.size, self.size))
        return img

# 2. Load Model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

# 3. Training Config
learning_rate = 2e-6
num_train_steps = 1000
batch_size = 1  # Reduce if you get OOM errors

# 4. Prepare Dataset
dataset = GhibliDataset("/content/convert_images")
train_dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True
)

# 5. Set Up Optimizer
optimizer = torch.optim.AdamW(
    pipe.unet.parameters(),
    lr=learning_rate,
)

# 6. Training Loop
pipe.unet.train()
for step, batch in enumerate(train_dataloader):
    if step >= num_train_steps:
        break

    # Convert images to latents
    latents = pipe.vae.encode(batch.to("cuda")).latent_dist.sample() * 0.18215

    # Sample noise
    noise = torch.randn_like(latents)
    bsz = latents.shape[0]

    # Sample a random timestep for each image
    timesteps = torch.randint(
        0, pipe.scheduler.num_train_timesteps, (bsz,), device=latents.device
    ).long()

    # Add noise to the latents
    noisy_latents = pipe.scheduler.add_noise(latents, noise, timesteps)

    # Get the text embedding for conditioning
    encoder_hidden_states = pipe.text_encoder(
        pipe.tokenizer(
            ["a ghibli style artwork"] * bsz,
            return_tensors="pt",
            padding="max_length",
            max_length=pipe.tokenizer.model_max_length,
            truncation=True,
        ).input_ids.to("cuda")
    )[0]

    # Predict the noise residual
    noise_pred = pipe.unet(noisy_latents, timesteps, encoder_hidden_states).sample

    # Compute loss
    loss = torch.nn.functional.mse_loss(noise_pred, noise)

    # Backpropagate
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss.item()}")

# 7. Save the trained model
pipe.save_pretrained("/content/ghibli_model")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

UnidentifiedImageError: cannot identify image file '/content/convert_images/IMG_3866.HEIC'